In [ ]:
wpip install nltk

In [ ]:
!pip install -U spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 27.5 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.5.4
    Uninstalling spacy-3.5.4:
      Successfully uninstalled spacy-3.5.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.5.0 requires spacy<3.6.0,>=3.5.0, but you have spacy 3.6.0 which is incompatible.


In [ ]:
!python -m spacy download en_core_web_md

2023-07-24 18:54:06.352795: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-24 18:54:07.584260: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 18.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [ ]:
!python -m spacy download en

2023-07-24 18:54:20.486640: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-24 18:54:21.427858: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 38.4 MB/s eta 0:00:00
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 3.5.0
    Uninstalling en-core-web-sm-3.5.0:
      Successfully uninstalled en-core-web-sm-3.5.0
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
import pandas as pd
import spacy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#df = pd.read_csv('Text_Similarity.csv')
df = pd.read_csv('/content/drive/MyDrive/Major Project/Text_Similarity.csv')

In [ ]:
df.head()

,text1,text2
0,broadband challenges tv viewing the number of ...,gardener wins double in glasgow britain s jaso...
1,rap boss arrested over drug find rap mogul mar...,amnesty chief laments war failure the lack of ...
2,player burn-out worries robinson england coach...,hanks greeted at wintry premiere hollywood sta...
3,hearts of oak 3-2 cotonsport hearts of oak set...,redford s vision of sundance despite sporting ...
4,sir paul rocks super bowl crowds sir paul mcca...,mauresmo opens with victory in la amelie maure...


In [ ]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('word_tokenize')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Error loading word_tokenize: Package 'word_tokenize' not
[nltk_data]     found in index


False

In [ ]:
nlp = spacy.load('en_core_web_md')
def preprocess_paragraph(paragraph):
    paragraph = paragraph.lower()
    paragraph = re.sub(r"[^a-zA-Z]", " ", paragraph)
    tokens = word_tokenize(paragraph)
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token not in stop_words]
    preprocessed_paragraph = " ".join(filtered_tokens)
    doc = nlp(preprocessed_paragraph)
    return doc

In [ ]:
df['Preprocessed_text1'] = df['text1'].apply(preprocess_paragraph)
df['Preprocessed_text2'] = df['text2'].apply(preprocess_paragraph)

In [ ]:
df.head()

,text1,text2,Preprocessed_text1,Preprocessed_text2
0,broadband challenges tv viewing the number of ...,gardener wins double in glasgow britain s jaso...,"(broadband, challenges, tv, viewing, number, e...","(gardener, wins, double, glasgow, britain, jas..."
1,rap boss arrested over drug find rap mogul mar...,amnesty chief laments war failure the lack of ...,"(rap, boss, arrested, drug, find, rap, mogul, ...","(amnesty, chief, laments, war, failure, lack, ..."
2,player burn-out worries robinson england coach...,hanks greeted at wintry premiere hollywood sta...,"(player, burn, worries, robinson, england, coa...","(hanks, greeted, wintry, premiere, hollywood, ..."
3,hearts of oak 3-2 cotonsport hearts of oak set...,redford s vision of sundance despite sporting ...,"(hearts, oak, cotonsport, hearts, oak, set, gh...","(redford, vision, sundance, despite, sporting,..."
4,sir paul rocks super bowl crowds sir paul mcca...,mauresmo opens with victory in la amelie maure...,"(sir, paul, rocks, super, bowl, crowds, sir, p...","(mauresmo, opens, victory, la, amelie, mauresm..."


In [ ]:
def calculate_sentence_embedding(paragraph):
    doc = nlp(paragraph)
    return doc.vector

In [ ]:
df['Embedding_text1'] = df['Preprocessed_text1'].apply(calculate_sentence_embedding)
df['Embedding_text2'] = df['Preprocessed_text2'].apply(calculate_sentence_embedding)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
def calculate_similarity(embedding1, embedding2):
    return cosine_similarity([embedding1], [embedding2])[0][0]

In [ ]:
df['Similarity_Score'] = df.apply(lambda row: calculate_similarity(row['Embedding_text1'], row['Embedding_text2']), axis=1)

In [ ]:
df.head(15)

,text1,text2,Preprocessed_text1,Preprocessed_text2,Embedding_text1,Embedding_text2,Similarity_Score
0,broadband challenges tv viewing the number of ...,gardener wins double in glasgow britain s jaso...,"(broadband, challenges, tv, viewing, number, e...","(gardener, wins, double, glasgow, britain, jas...","[-0.4131709, 0.34337187, -1.4226754, 0.3016963...","[-1.3108351, 0.44678646, -0.48292035, 0.924576...",0.782756
1,rap boss arrested over drug find rap mogul mar...,amnesty chief laments war failure the lack of ...,"(rap, boss, arrested, drug, find, rap, mogul, ...","(amnesty, chief, laments, war, failure, lack, ...","[-1.9650658, -0.057604477, -1.0913287, -0.0512...","[-0.87831855, 0.60816604, -2.0568476, -0.10608...",0.763699
2,player burn-out worries robinson england coach...,hanks greeted at wintry premiere hollywood sta...,"(player, burn, worries, robinson, england, coa...","(hanks, greeted, wintry, premiere, hollywood, ...","[-0.5034084, 0.7766004, -0.6720093, -0.1318602...","[-1.0789757, 0.016381567, -1.5343, -0.97832906...",0.829972
3,hearts of oak 3-2 cotonsport hearts of oak set...,redford s vision of sundance despite sporting ...,"(hearts, oak, cotonsport, hearts, oak, set, gh...","(redford, vision, sundance, despite, sporting,...","[-1.8217907, 0.7138336, -0.0002754031, 0.75650...","[-0.8257285, 0.77039766, -2.0269697, -0.354338...",0.695401
4,sir paul rocks super bowl crowds sir paul mcca...,mauresmo opens with victory in la amelie maure...,"(sir, paul, rocks, super, bowl, crowds, sir, p...","(mauresmo, opens, victory, la, amelie, mauresm...","[-0.780121, 0.6844035, -1.6178967, -0.13017784...","[-0.7605208, 0.7402197, -0.7952503, 0.28600857...",0.889604
5,india to deport bollywood actress india has or...,fosters buys stake in winemaker australian bre...,"(india, deport, bollywood, actress, india, ord...","(fosters, buys, stake, winemaker, australian, ...","[-0.60054106, 0.64595205, -1.0528442, -0.31953...","[-1.2402834, -0.37793618, -1.9289837, 0.338310...",0.845752
6,mutant book wins guardian prize a book about t...,jp morgan admits us slavery links thousands of...,"(mutant, book, wins, guardian, prize, book, ev...","(jp, morgan, admits, us, slavery, links, thous...","[-0.918563, -0.7966027, -0.2839613, -1.0063003...","[-1.9456744, -0.39453277, -1.4665296, -0.19474...",0.797540
7,aids and climate top davos agenda climate chan...,howl helps boost japan s cinemas japan s box o...,"(aids, climate, top, davos, agenda, climate, c...","(howl, helps, boost, japan, cinemas, japan, bo...","[-0.9041047, 0.52750033, -1.3378434, 0.3548534...","[-1.8236586, -0.8764372, -1.7492514, -0.802548...",0.802771
8,kennedy predicts bigger turnout voters pent ...,ocean s twelve raids box office ocean s twelve...,"(kennedy, predicts, bigger, turnout, voters, p...","(ocean, twelve, raids, box, office, ocean, twe...","[-0.42047408, 0.4145867, -1.9834875, 0.7388388...","[-1.0918648, -0.13607423, -1.2265658, -0.28855...",0.767788
9,carry on star patsy rowlands dies actress pats...,uk broadband gets speed injection broadband s ...,"(carry, star, patsy, rowlands, dies, actress, ...","(uk, broadband, gets, speed, injection, broadb...","[-1.4485404, -0.019717734, -1.5869294, -1.1226...","[-0.69785666, 0.59646106, -1.652467, 0.3620873...",0.706233


In [ ]:
df['Similarity_Score'].describe()

count    3000.000000
mean        0.806955
std         0.078377
min         0.396308
25%         0.759912
50%         0.811975
75%         0.864305
max         1.000000
Name: Similarity_Score, dtype: float64

In [ ]:
df['text1'].head()

0    broadband challenges tv viewing the number of ...
1    rap boss arrested over drug find rap mogul mar...
2    player burn-out worries robinson england coach...
3    hearts of oak 3-2 cotonsport hearts of oak set...
4    sir paul rocks super bowl crowds sir paul mcca...
Name: text1, dtype: object